In [ ]:
# imports necesarios
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
#cargamos los datos de entrada
data = pd.read_csv("./articulos_ml.csv")
#veamos cuantas dimensiones y registros contiene
data.shape

In [ ]:
data.head()

In [ ]:
# Ahora veamos algunas estadísticas de nuestros datos
data.describe()

In [ ]:
# Visualizamos rápidamente las caraterísticas de entrada
data.drop(['Title', 'url', 'Elapsed days'], axis=1).hist()
plt.show()

In [ ]:
# Vamos a RECORTAR los datos en la zona donde se concentran más los puntos
# esto es en el eje X: entre 0 y 3.500
# y en el eje Y: entre 0 y 80.000
filtered_data = data[(data['Word count'] <= 3500) & (data['# Shares'] <= 80000)]
colores=['orange','blue']
tamanios=[30,60]
f1 = filtered_data['Word count'].values
f2 = filtered_data['# Shares'].values
# Vamos a pintar en colores los puntos por debajo y por encima de la media de Cantidad de Palabras
asignar=[]
for index, row in filtered_data.iterrows():
    if(row['Word count']>1808):
        asignar.append(colores[0])
    else:
         asignar.append(colores[1])

plt.scatter(f1, f2, c=asignar, s=tamanios[0])
plt.show()


In [ ]:
# Asignamos nuestra variable de entrada X para entrenamiento y las etiquetas Y.
dataX = filtered_data[["Word count"]]
X_train = np.array(dataX)
y_train = filtered_data['# Shares'].values

# Creamos el objeto de Regresión Linear
regr = linear_model.LinearRegression()

# Entrenamos nuestro modelo
regr.fit(X_train, y_train)

# Hacemos las predicciones que en definitiva una línea (en este caso, al ser 2D)
y_pred = regr.predict(X_train)

# Veamos los coeficientes obtenidos, en nuestro caso, serán la Tangente
print('Coefficients: \n', regr.coef_)
# Este es el valor donde corta el eje Y (en X=0)
print('Independent term: \n', regr.intercept_)
# Error Cuadrado Medio
print("Mean squared error: %.2f" % mean_squared_error(y_train, y_pred))
# Puntaje de Varianza. El mejor puntaje es un 1.0
print('Variance score: %.2f' % r2_score(y_train, y_pred))


In [ ]:
# Graficamos los datos
plt.scatter(X_train, y_train, color='blue', label="Datos reales")
plt.plot(X_train, y_pred, color='red', linewidth=2, label="Regresión lineal")

plt.xlabel("Word count")
plt.ylabel("# Shares")
plt.legend()
plt.show()

In [ ]:
#Vamos a comprobar:
# Quiero predecir cuántos "Shares" voy a obtener por un artículo con 2.000 palabras,
# según nuestro modelo, hacemos:
y_Dosmil = regr.predict([[2000]])
print(int(y_Dosmil))

In [ ]:
# Vamos a intentar mejorar el Modelo con una dimensión más:
# Creamos una nueva variable que es la suma de enlaces, comentarios e imágenes
suma = (filtered_data["# of Links"] + 
        filtered_data['# of comments'].fillna(0) + 
        filtered_data['# Images video'])

# Creamos el nuevo DataFrame con dos características
dataX2 = pd.DataFrame()
dataX2["Word count"] = filtered_data["Word count"]
dataX2["suma"] = suma

# Convertimos a arrays de NumPy para el entrenamiento
XY_train = np.array(dataX2)
z_train = filtered_data['# Shares'].values

In [ ]:
 # Creamos un nuevo objeto de Regresión Lineal
regr2 = linear_model.LinearRegression()

# Entrenamos el modelo, esta vez, con 2 dimensiones
# obtendremos 2 coeficientes, para graficar un plano
regr2.fit(XY_train, z_train)

# Hacemos la predicción con la que tendremos puntos sobre el plano hallado
z_pred = regr2.predict(XY_train)

# Los coeficientes
print('Coefficients: \n', regr2.coef_)
# Error cuadrático medio
print("Mean squared error: %.2f" % mean_squared_error(z_train, z_pred))
# Evaluamos el puntaje de varianza (siendo 1.0 el mejor posible)
print('Variance score: %.2f' % r2_score(z_train, z_pred))


In [ ]:
fig = plt.figure(figsize=(16, 10))  
ax = fig.add_subplot(111, projection='3d')

xx, yy = np.meshgrid(np.linspace(0, 3500, num=50), np.linspace(0, 60, num=50))

nuevoX = regr2.coef_[0] * xx
nuevoY = regr2.coef_[1] * yy
z = nuevoX + nuevoY + regr2.intercept_

ax.plot_surface(xx, yy, z, alpha=0.5, cmap='coolwarm', shade=True)

ax.scatter(XY_train[:, 0], XY_train[:, 1], z_train, c='blue', s=50, label="Datos reales", alpha=0.85)

z_pred = regr2.predict(XY_train)
ax.scatter(XY_train[:, 0], XY_train[:, 1], z_pred, c='red', s=55, label="Predicciones", alpha=0.9)

ax.view_init(elev=20, azim=45)

ax.grid(True)

ax.set_xlabel('Cantidad de Palabras', fontsize=14, labelpad=20)
ax.set_ylabel('Suma de Enlaces, Comentarios e Imágenes', fontsize=14, labelpad=20)
ax.set_zlabel('Compartido en Redes', fontsize=14, labelpad=20)
ax.set_title('Regresión Lineal con Múltiples Variables', fontsize=16, pad=25)

ax.legend(fontsize=14)

plt.show()

In [ ]:
# Datos de entrada para predecir
nuevos_datos = np.array([[2000, 10 + 4 + 6]])

# Predicción con el modelo entrenado
z_Dosmil = regr2.predict(nuevos_datos)

# Imprimimos la cantidad estimada de Shares
print(int(z_Dosmil[0]))  # Convertimos el resultado a entero